# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [98]:
# Load the libraries as required.
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score
from sklearn.linear_model import Lasso, Ridge


In [99]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [100]:
feat_col = fires_dt.columns.difference(['area'])

df_feat = fires_dt[feat_col]

df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   day      517 non-null    object 
 3   dc       517 non-null    float64
 4   dmc      517 non-null    float64
 5   ffmc     517 non-null    float64
 6   isi      517 non-null    float64
 7   month    517 non-null    object 
 8   rain     517 non-null    float64
 9   rh       517 non-null    int64  
 10  temp     517 non-null    float64
 11  wind     517 non-null    float64
dtypes: float64(7), int64(3), object(2)
memory usage: 48.6+ KB


In [101]:
df_pred = fires_dt[['area']]

df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   area    517 non-null    float64
dtypes: float64(1)
memory usage: 4.2 KB


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [102]:
numeric_cols = df_feat.select_dtypes(include=['int64','float64']).columns.to_list()
category_cols = df_feat.select_dtypes(include=['object']).columns.to_list()

preproc1 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(),  numeric_cols),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), category_cols), 
    ]
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [103]:
preproc2 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', PowerTransformer(),  numeric_cols),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), category_cols), 
    ]
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [104]:
# Pipeline A = preproc1 + baseline

pipeA = Pipeline(
    [
        ('preprocessing', preproc1), 
        ('regressor', Lasso())
    ]
)

pipeA

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'dc',
                                                   'dmc', 'ffmc', 'isi', 'rain',
                                                   'rh', 'temp', 'wind']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['day', 'month'])])),
                ('regressor', Lasso())])

In [105]:
# Pipeline B = preproc2 + baseline
pipeB = Pipeline(
    [
        ('preprocessing', preproc2), 
        ('regressor', Lasso())
    ]
)

pipeB

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  PowerTransformer(),
                                                  ['coord_x', 'coord_y', 'dc',
                                                   'dmc', 'ffmc', 'isi', 'rain',
                                                   'rh', 'temp', 'wind']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['day', 'month'])])),
                ('regressor', Lasso())])

In [106]:
# Pipeline C = preproc1 + advanced model

pipeC = Pipeline(
    [
        ('preprocessing', preproc1), 
        ('regressor', Ridge())
    ]
)

pipeC


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  StandardScaler(),
                                                  ['coord_x', 'coord_y', 'dc',
                                                   'dmc', 'ffmc', 'isi', 'rain',
                                                   'rh', 'temp', 'wind']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['day', 'month'])])),
                ('regressor', Ridge())])

In [107]:
# Pipeline D = preproc2 + advanced model

pipeD = Pipeline(
    [
        ('preprocessing', preproc2), 
        ('regressor', Ridge())
    ]
)

pipeD


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numeric_transfomer',
                                                  PowerTransformer(),
                                                  ['coord_x', 'coord_y', 'dc',
                                                   'dmc', 'ffmc', 'isi', 'rain',
                                                   'rh', 'temp', 'wind']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['day', 'month'])])),
                ('regressor', Ridge())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [108]:
X = fires_dt[feat_col]
Y = fires_dt['area']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [109]:
grid_cv_A = GridSearchCV(
    estimator=pipeA, 
    param_grid={'regressor__alpha': [1,2,3,4,5]},
    scoring = 'r2',
    cv = 5,
    )

grid_cv_A.fit(X_train, Y_train)

resA = grid_cv_A.cv_results_
resA = pd.DataFrame(resA)

resA

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005779,0.000993,0.002936,0.000738,1,{'regressor__alpha': 1},0.025876,-0.313685,-0.127547,0.003202,-0.004991,-0.083429,0.126940,5
1,0.004572,0.001016,0.002119,0.000231,2,{'regressor__alpha': 2},0.024312,-0.207882,-0.005796,-0.002939,0.009002,-0.036660,0.086266,4
2,0.004193,0.000769,0.002376,0.000529,3,{'regressor__alpha': 3},0.017508,-0.150510,0.011819,-0.009293,-0.002073,-0.026510,0.062732,1
3,0.004651,0.000581,0.002332,0.000646,4,{'regressor__alpha': 4},0.008123,-0.126037,0.015083,-0.010641,-0.021286,-0.026952,0.051212,2
4,0.004149,0.000801,0.002485,0.000428,5,{'regressor__alpha': 5},0.001756,-0.124725,0.006654,-0.010641,-0.033647,-0.032121,0.048354,3


In [110]:
grid_cv_B = GridSearchCV(
    estimator=pipeB, 
    param_grid={'regressor__alpha': [1,2,3,4,5]},
    scoring = 'r2',
    cv = 5,
    )

grid_cv_B.fit(X_train, Y_train)

resB = grid_cv_B.cv_results_
resB = pd.DataFrame(resB)

resB

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018715,0.004022,0.002516,0.000463,1,{'regressor__alpha': 1},0.028299,-0.336742,-0.057602,0.002343,-0.004193,-0.073579,0.134512,5
1,0.016017,0.000670,0.002504,0.000551,2,{'regressor__alpha': 2},0.024086,-0.224637,-0.009795,-0.003851,0.007998,-0.041240,0.092426,4
2,0.016529,0.000923,0.002199,0.000400,3,{'regressor__alpha': 3},0.016939,-0.165788,0.012134,-0.009842,-0.004679,-0.030247,0.068504,1
3,0.016054,0.001196,0.002304,0.000600,4,{'regressor__alpha': 4},0.006744,-0.140832,0.014125,-0.010641,-0.025232,-0.031167,0.056518,2
4,0.016251,0.000843,0.002815,0.000615,5,{'regressor__alpha': 5},0.000714,-0.137138,0.005062,-0.010641,-0.035064,-0.035413,0.052736,3


In [111]:
grid_cv_C = GridSearchCV(
    estimator=pipeC, 
    param_grid={'regressor__alpha': [1,2,3,4,5]},
    scoring = 'r2',
    cv = 5,
    )

grid_cv_C.fit(X_train, Y_train)

resC = grid_cv_C.cv_results_
resC = pd.DataFrame(resC)

resC

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008112,0.005029,0.002803,0.000402,1,{'regressor__alpha': 1},-0.022660,-0.715965,-0.563624,-0.008575,-0.199253,-0.302015,0.287924,5
1,0.004790,0.000491,0.002708,0.000608,2,{'regressor__alpha': 2},-0.023397,-0.691850,-0.562495,-0.008297,-0.186111,-0.294430,0.281732,4
2,0.004782,0.000448,0.002842,0.000206,3,{'regressor__alpha': 3},-0.023225,-0.675031,-0.557548,-0.007948,-0.176735,-0.288097,0.276899,3
3,0.004419,0.000514,0.003016,0.000027,4,{'regressor__alpha': 4},-0.022635,-0.661526,-0.551675,-0.007565,-0.168859,-0.282452,0.272818,2
4,0.004889,0.000441,0.002604,0.000585,5,{'regressor__alpha': 5},-0.021835,-0.650009,-0.545688,-0.007175,-0.161856,-0.277313,0.269259,1


In [112]:
grid_cv_D = GridSearchCV(
    estimator=pipeD, 
    param_grid={'regressor__alpha': [1,2,3,4,5]},
    scoring = 'r2',
    cv = 5,
    )

grid_cv_D.fit(X_train, Y_train)

resD = grid_cv_D.cv_results_
resD = pd.DataFrame(resD)

resD

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018885,0.001876,0.002440,0.000537,1,{'regressor__alpha': 1},-0.011426,-0.721833,-0.165992,-0.004991,-0.193771,-0.219603,0.262744,5
1,0.020620,0.000570,0.002902,0.000198,2,{'regressor__alpha': 2},-0.012594,-0.686859,-0.154758,-0.005144,-0.176688,-0.207209,0.249981,4
2,0.020145,0.001493,0.003002,0.000004,3,{'regressor__alpha': 3},-0.012916,-0.665813,-0.147531,-0.005185,-0.164816,-0.199252,0.242455,3
3,0.018963,0.000291,0.002902,0.000487,4,{'regressor__alpha': 4},-0.012772,-0.650924,-0.142273,-0.005139,-0.155423,-0.193306,0.237256,2
4,0.020210,0.001469,0.002965,0.000068,5,{'regressor__alpha': 5},-0.012362,-0.639343,-0.138147,-0.005039,-0.147492,-0.188477,0.233306,1


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.